In [228]:
import json
import google.generativeai as palm
import numpy as np
import spacy

nlp = spacy.load('en_core_web_lg')

my_config = json.load(open('config.json'))

palm.configure(api_key=my_config['palm_api_key'])

In [229]:
text_model = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods][0].name
embeddings_model = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods][0].name

In [230]:
tools = json.load(open('tools.json'))
models = palm.list_models()
example_prompts = json.load(open('example_prompts.json'))["examples"]

In [231]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.tuning']

def load_creds():
    """Converts `oauth-client-id.json` to a credential object.
    
    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_56510766963.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# load_creds()

In [232]:
arg_prompt = """
I have a argument for an function call. I want to summarize it and get its data type and possible values.
For Example:

Input:{"name": "ticket.severity","description": "Filters for tickets with any of the provided severities. Allowed values: blocker, high, low, medium","type": "array of strings"}
Ouput: {"desc": "Filters for tickets with any of the provided severities", "type": "Array[String]", "allowed": ["blocker", "high", "low", "medium"]}

Input:{"name": "query","description": "The search string, could be for example customer's name, part name, user name.","type": "string"}
Output: {"desc": "The search string, could be for example customer's name, part name, user name.", "type": "String"}

Input: {"name": "limit","description": "The maximum number of works to return. The default is '50'","type": "integer (int32)"}
Output: {"desc": "The maximum number of works to return.", "type": "int32"}

Input: {"name": "applies_to_part", "description": "Filters for work belonging to any of the provided parts", "type": "array of strings", "example": [ "FEAT-123", "ENH-123", "PROD-123", "CAPL-123" ]}
Output: {"desc": "Filters for work belonging to any of the provided parts.", "type": "Array[String]", "example": ["FEAT-123", "ENH-123", "PROD-123", "CAPL-123"]}

solve for the following argument ensuring output is valid json:
Input: %s
Output:
"""

arguments_description = {}

if os.path.exists('refined_arguments_description.json'):
    arguments_description = json.load(open('refined_arguments_description.json'))
else:
    for tool in tools["tools"]:
        arguments_description[tool["name"]] = []
        for argument in tool.get("arguments" ,[]):
            output = palm.generate_text(
                model=text_model,
                prompt=arg_prompt % argument,
                temperature=0,
                max_output_tokens=800,
            )
            arguments_description[tool["name"]].append({argument["name"]: eval(output.result)}) 
    json.dump(arguments_description, open('refined_arguments_description.json', 'w'))

In [233]:
def get_tools_description(tools, argument_descriptions):
    tools_description = ""
    for tool in tools["tools"]:
        tools_description += "\n" + f"{tool['name']}:{tool['description'].split('.')[0]}" 
        for argument in argument_descriptions[tool["name"]]:
            tools_description += " with args:"
            for arg, props in argument.items():
                tools_description += f"\n\t{arg}:{props['desc'].split('.')[0]}"
                # if 'example' in props.keys():
                #     tools_description += f" like: {props['example'][:2]}"
    return tools_description

print(get_tools_description(tools, arguments_description))


works_list:Returns a list of work items matching the request with args:
	applies_to_part:Filters for work belonging to any of the provided parts with args:
	created_by:Filters for work created by any of these users with args:
	owned_by:Filters for work owned by any of these users with args:
	issue.rev_orgs:Filters for issues with any of the provided Rev organizations with args:
	issue.priority:Filters for issues with any of the provided priorities with args:
	limit:The maximum number of works to return with args:
	stage.name:Filters for records in the provided stage(s) by name with args:
	ticket.needs_response:Filters for tickets that need a response with args:
	ticket.severity:Filters for tickets with any of the provided severities with args:
	ticket.source_channel:Filters for tickets with any of the provided source channels with args:
	types:Filters for work of the provided types
summarize_objects:Summarizes a list of objects with args:
	objects:List of objects to summarize
prioriti

In [234]:
segmentation_prompt = """
You have sentence which contains a task statement. You need to perform coreference resolution and split it into multiple sentences containing simpler atomic subtasks without fabricating any information. E.g.:
Input: Retrieve all tasks resembling TKT-123, condense the information, generate problem reports based on the summary, and arrange them in order of priority
Output: ["Get tasks resembling TKT-123","Summarize the tasks resembling TKT-123","Generate reports from summarized tasks","Prioritize the tasks resembling TKT-123"]

Input: Post an AI generated image on twitter. Monitor the number of likes and retweets. Perform analysis on the collected metrics.
Output: ["Post AI generated image on twitter","Monitor the AI generated post on twitter","Analyze monitored metrics"]

Input: Generate product reccommendations for the user User-123 based on the summary of all posts made by them.
Output: ["Get all posts by User-123","Summarize the posts by User-123","Generate product reccommendations from summarized posts"]  

Input: Add all issues P0 issues by User-123 to my sprint. Also create their summary.
Output: ["Get all P0 issues by User-123","Add P0 issues by User-123 to sprint","Summarize the P0 issues by User-123"]

Input: Could you get all my high priority work items?
Output: ["Get all my work items assigned high priority"]

Input: %s
Output:
"""

In [235]:
response = palm.generate_text(
    model=text_model,
    prompt=segmentation_prompt % "Summarize high severity tickets from the customer UltimateCustomer"
)

In [236]:
response.result

'["Get high severity tickets from the customer UltimateCustomer","Summarize the high severity tickets from the customer UltimateCustomer"]'

In [237]:
for example in example_prompts:
    response = palm.generate_text(
        model=text_model,
        prompt=segmentation_prompt % example,
        temperature=0,
        max_output_tokens=800,
    )
    print(f"Input: {example}\nOutput: {response.result}\n")

Input: Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1
Output: ["Get work items similar to don:core:dvrv-us-1:devo/0:issue/1","Summarize the work items"]

Input: What is the meaning of life?
Output: ["Get the meaning of life"]

Input: Prioritize my P0 issues and add them to the current sprint
Output: ["Prioritize my P0 issues","Add my P0 issues to the current sprint"]

Input: Summarize high severity tickets from the customer UltimateCustomer
Output: ["Get high severity tickets from the customer UltimateCustomer","Summarize the high severity tickets from the customer UltimateCustomer"]

Input: What are my all issues in the triage stage under part FEAT-123? Summarize them.
Output: ["Get all my issues in the triage stage under part FEAT-123","Summarize the issues in the triage stage under part FEAT-123"]

Input: List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them.
Output: ["Get all high severity tickets coming in from 

In [245]:
tool_getter_prompt = """
You a work management bot for a company with access to the following tools:
You need to return a tool (if any) that will complete the primary objective of the task using the tool descriptions. Return the empty string if not available Eg:
Tools:
A: A tool that As
B: A tool that Bs
Input: A the task item
Output: A
Input: B the task item
Output: B
Input: C the task item
Output: None

Tools: %s
Input: %s
Ouput:
"""

In [256]:
tools_description = get_tools_description(tools, arguments_description)

example_tasks = ["Get the meaning of life"]
example_tasks = eval(response.result)
# example_tasks = [
#     "Get all high severity tickets coming in from slack from customer Cust123",
#     "Summarize the high severity tickets coming in from slack from customer Cust123",
# ]
# example_tasks = ["Prioritize my P0 issues", "Add my P0 issues to the current sprint"]


def get_tools_for_tasks(tasks):
    output = []
    for task in tasks:
        response = palm.generate_text(
            model=text_model,
            prompt=tool_getter_prompt % (tools_description, task),
            temperature=0,
            max_output_tokens=800,
        )
        result = response.result
        if result == "None":
            return []
        output.append((task, response.result))
    return output

# print(tool_getter_prompt % (tools_description, example_tasks[0]))
task_and_tool = get_tools_for_tasks(example_tasks)

In [257]:
print(task_and_tool)

[('Get all work items similar to TKT-123', 'get_similar_work_items'), ('Summarize the work items similar to TKT-123', 'summarize_objects'), ('Create issues from summarized work items', 'create_actionable_tasks_from_text'), ('Prioritize the issues created from summarized work items', 'prioritize_objects')]


In [258]:
class KnowledgeItem:
    description: str
    tool: str

    def __init__(self, description: str, tool: str) -> None:
        self.description = description
        self.tool = tool

    def summarize(self) -> str:
        return self.description + ":" + self.tool

    def __str__(self) -> str:
        return f"Know <{self.description} from [{self.tool}]>"

    def __repr__(self) -> str:
        return str(self)


def get_base_knowledge(tools, arguments_description):
    knowledge = []

    l = list(arguments_description.keys())

    for tool in arguments_description:
        if len(arguments_description[tool]) == 0:
            tool_names = [t["name"] for t in tools["tools"]]
            index = tool_names.index(tool)
            tool_description = tools["tools"][index]["description"].split("Returns ")[1]
            knowledge.append(KnowledgeItem(tool_description, tool))

    return knowledge


base_knowledge = get_base_knowledge(tools, arguments_description)

#### Fine tune on more examples like the ones below to improve accuracy

In [259]:
subtask_solution_prompt = """
You're a issue management bot. Given a tool and directive, you look for the arguments in the Past Actions/Knowledge or the directive. If you find them, you return the arguments. If not, you return the missing action. Eg:
Directive:Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
\tobjects (Array[Object]): The objects to summarize
Past Actions/Knowledge:
\tPrioritize the work items similar to TKT-123:prioritize_work_items
\tGet all similar work items to TKT-123:get_similar_work_items
\tthe ID of the current User:who_am_i
\tthe current sprint:get_current_sprint
Thought:I have work items similar to TKT-123 so I can summarize them.
Output:{"result":[("objects",["get_similar_work_items"])],"missing_action":""}

Directive:Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
\tapplies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
\towned_by (Array[String]): Filters for work owned by any of these users
\tcreated_by (Array[String]): Filters for work created by any of these users
\tissue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3']
Past Actions/Knowledge:
\tthe ID of the current User:who_am_i
\tthe current sprint:get_current_sprint
Thought:The user wants their P0 issues on part DEV-PART-123
Output:{"result":[("owned_by",["who_am_i"]),("issue.priority",["p0"]),("applies_to_part",["DEV-PART-123"])],"missing_action":""}

Directive:Summarize my P0 issues
Tool:summarize_objects with args:
\tobjects (Array[Object]):The objects to summarize
Past Actions/Knowledge:
\tthe ID of the current User: who_am_i
\tthe current sprint: get_current_sprint
Thought: I don't know what P0 issues are.
Output: {"result":[],"missing_action":"Get my P0 issues"}
"""
print(subtask_solution_prompt)


You're a issue management bot. Given a tool and directive, you look for the arguments in the Past Actions/Knowledge or the directive. If you find them, you return the arguments. If not, you return the missing action. Eg:
Directive:Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
	objects:The objects to summarize
Past Actions/Knowledge:
	Prioritize the work items similar to TKT-123:prioritize_work_items
	Get all similar work items to TKT-123:get_similar_work_items
	the ID of the current User:who_am_i
	the current sprint:get_current_sprint
Thought:I have work items similar to TKT-123 so I can summarize them.
Output:{"result":[("objects","get_similar_work_items")],"missing_action":""}

Directive:Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
	applies_to_part: Filters for work belonging to any of the provided parts.
	owned_by: Filters for work owned by any of these users
	created_by: Filters for work created by any of these users
	issue.priori

In [260]:
def elaborate_args(args: list[dict]):
    response = ""
    primary_count = 0
    for arg in args:
        for name, props in arg.items():
            if primary_count < 3:
                response += f"{name}*:{props['desc']}"
                primary_count += 1
            else:
                response += f"{name}:{props['desc']}"
            if 'allowed' in props.keys():
                response += f" allowing: {props['allowed']}"
            response += "\n\t"

    return response


def get_instruction_prompt(instruction, arguments_description, knowledge):
    directive = instruction[0]
    tool_to_be_used = instruction[1]

    tool_arguments = arguments_description[tool_to_be_used]

    prompt = f"give response for:\n"
    prompt += f"Directive:{directive}'\nTool: {tool_to_be_used} with args:\n\t{elaborate_args(tool_arguments)}"
    
    prompt += "\nPast Actions/Knowledge:"
    for knowledge_item in knowledge[::-1]:
        prompt += f"\n\t{knowledge_item.summarize()}"

    prompt += "\nOutput:"

    return prompt

In [265]:
from collections import deque

instructions = deque(task_and_tool[1:])


def complete_task(instructions: deque, tools, arguments_description):
    knowledge = get_base_knowledge(tools, arguments_description)

    total_len = 0
    steps = 0

    while len(instructions) > 0:
        if steps > 6:
            break
        steps += 1
        instruction = instructions[0]
        response = palm.generate_text(
            model=text_model,
            prompt=subtask_solution_prompt
            + get_instruction_prompt(instruction, arguments_description, knowledge),
            temperature=0,
            # max_output_tokens=800,
        )
        print(f"Input: {instruction}\nOutput: {response.result}")
        response = eval(response.result)

        tokens = palm.count_text_tokens(
            model=text_model,
            prompt=subtask_solution_prompt
            + get_instruction_prompt(instruction, arguments_description, knowledge),
        )
        total_len += tokens['token_count']

        if len(response["missing_action"]) > 0:
            print(f"Missing action: {response['missing_action']}")

            tool_for_missing_action = get_tools_for_tasks([response["missing_action"]])

            print(f"Tool for missing action: {tool_for_missing_action}")

            if len(tool_for_missing_action) > 0:
                instructions.appendleft((response["missing_action"], tool_for_missing_action[0][1]))
        else:
            print(f"Result: {response['result']}")
            instructions.popleft()
            knowledge.append(KnowledgeItem(instruction[0], instruction[1]))
        print()
    
    print(f"Total tokens: {total_len}")
    return knowledge


# print(instructions)

complete_task(instructions, tools, arguments_description)

Input: ('Summarize the work items similar to TKT-123', 'summarize_objects')
Output: {"result":[("objects","get_similar_work_items")],"missing_action":""}
Result: [('objects', 'get_similar_work_items')]

Input: ('Create issues from summarized work items', 'create_actionable_tasks_from_text')
Output: {"result":[("text","summarize_objects")],"missing_action":""}
Result: [('text', 'summarize_objects')]

Input: ('Prioritize the issues created from summarized work items', 'prioritize_objects')
Output: {"result":[("objects","create_actionable_tasks_from_text")],"missing_action":""}
Result: [('objects', 'create_actionable_tasks_from_text')]

Total tokens: 1858


[Know <the ID of the current sprint from [get_sprint_id]>,
 Know <the ID of the current user from [who_am_i]>,
 Know <Summarize the work items similar to TKT-123 from [summarize_objects]>,
 Know <Create issues from summarized work items from [create_actionable_tasks_from_text]>,
 Know <Prioritize the issues created from summarized work items from [prioritize_objects]>]

In [297]:
my_base_examples = [
    """
Directive: Summarize the work items similar to TKT-123
Tool:summarize_objects with args:
	objects (Array[Object]):The objects to summarize
Past Actions/Knowledge:
	Prioritize the work items similar to TKT-123: prioritize_work_items
	Get all similar work items to TKT-123: get_similar_work_items
	the ID of the current User: who_am_i
	the current sprint:get_current_sprint
Output:{"result":[("objects",["get_similar_work_items"])],"missing_action":""}
""",
    """
Directive: Get my P0 issues on part DEV-PART-123
Tool:get_items with args:
	applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
	owned_by (Array[String]): Filters for work owned by any of these users
	created_by (Array[String]): Filters for work created by any of these users 
	issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
	the ID of the current User: who_am_i
Output:{"result": [("owned_by",["who_am_i"]),("issue.priority",["p0"]),("applies_to_part",["DEV-PART-123"])],"missing_action": ""}
""",
    """
Directive: Summarize my P0 issues
Tool:summarize_objects with args:
	objects (Array[Object]):The objects to summarize
Past Actions/Knowledge:
	the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
    """
Directive: Prioritize my P0 issues
Tool:prioritize_objects with args:
	objects (Array[Object]): The objects to summarize
Past Actions/Knowledge:
	the ID of the current User: who_am_i
Output: {"result": [],"missing_action": "Get my P0 issues"}
""",
    """
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
    to (String): The email address of the recipient
    from (String): The email address of the sender
    subject (String): The subject of the email (optional)
    body (String): The body of the email
Past Actions/Knowledge:
    Get user details of User-123: get_user_details
Output: {"result": [], "missing_action": "Summarize issue I-123"}
""",
    """
Directive: Get all issues
Tool:get_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
Output:{"result": [],"missing_action": ""}
""",
    """
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
    to (String): The email address of the recipient
    subject (String): The subject of the email (optional)
    body (String): The body of the email
Past Actions/Knowledge:
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
    """
Directive:Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
	applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
	owned_by (Array[String]): Filters for work owned by any of these users
	created_by (Array[String]): Filters for work created by any of these users 
	issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
	the ID of the current User: who_am_i
Output:{"result": [],"missing_action": "Get all parts containing the word "AI""}
""",
    """
Directive: Get my P0 issues on all objects containing the word "AI"
Tool:get_items with args:
	applies_to_part (String): Filters for work belonging to any of the provided parts.
	owned_by (String): Filters for work owned by any of these users
	created_by (String): Filters for work created by any of these users 
	issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
    Get all parts containing the word "AI": get_objects_by_keyword
	the ID of the current User: who_am_i
Output:{"result": [("owned_by","who_am_i"),("issue.priority","p0"),("applies_to_part","get_objects_by_keyword")],"missing_action": ""}
""",
    """
Directive: Send an email to User-123 regarding issue I-123.
Tool: send_email with args:
    to (String): The email address of the recipient
    from (String): The email address of the sender
    subject (String): The subject of the email (optional)
    body (String): The body of the email
Past Actions/Knowledge:
    Get user details of User-123: get_user_details
    Summarize issue I-123: summarize_objects
    the ID of the current User: who_am_i
Output: {"result": [("to", "get_user_details"), ("from", "who_am_i"), ("subject", "Regarding issue I-123"), ("body", "summarize_objects")],"missing_action": ""}
""",
    """
Directive: Dial up the user User-123
Tool: call_user with args:
    contact_details (Object): The contact details of the user
Past Actions/Knowledge:
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get user details of User-123"}
""",
    """
Directive: Dial up the user User-123
Tool: call_user with args:
    contact_details (Object): The contact details of the user
Past Actions/Knowledge:
    Get user details of User-123: get_user_details
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details")], "missing_action": ""}
""",
    """
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
    contact_details (Object): The contact details of the user
    amount (Number): The amount to pay the user
Past Actions/Knowledge:
    Get balance amount of User-123: get_user_balance
    Get user details of User-123: get_user_details
    the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", "get_user_balance")], "missing_action": ""}
""",
    """
Directive: Pay user User-123 his balance amount
Tool: pay_user with args:
    contact_details (Object): The contact details of the user
    amount (Number): The amount to pay the user
Past Actions/Knowledge:
    Get user details of User-123: get_user_details
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get balance amount of User-123"}
""",
    """
Directive: Pay user User-123 his $100
Tool: pay_user with args:
    contact_details (Object): The contact details of the user
    amount (Number): The amount to pay the user
Past Actions/Knowledge:
    Get user details of User-123: get_user_details
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("contact_details", "get_user_details"), ("amount", 100)], "missing_action": ""}
""",
    """
Directive: Close all unresolved issues
Tool: close_issues with args:
    issues (Array[Object]): List of issues to close
Past Actions/Knowledge:
Output: {"result": [], "missing_action": "Get the list of unresolved issues"}
""",
    """
Directive: Move all tasks to the next sprint
Tool: move_issues with args:
    issues (Array[Object]): List of issues to move
    destination_sprint (String): The sprint to move the issues to
Past Actions/Knowledge:
    the ID of the current User: who_am_i
    the next sprint: get_next_sprint
Output: {"result": [], "missing_action": "Get the list of all tasks"}
""",
    """
Directive: Move all unresolved issues to the next sprint
Tool: move_issues with args:
    issues (Array[Object]): List of issues to move
    destination_sprint (String): The sprint to move the issues to
Past Actions/Knowledge:
    Get List of unresolved issues: get_unresolved_issues
    the ID of the current User: who_am_i
    the next sprint: get_next_sprint
Output: {"result": [("issues", ["get_unresolved_issues"]), ("destination_sprint", "get_next_sprint")], "missing_action": ""}
""",
    """
Directive: Get my issues on all objects made by MAN-123
Tool:get_items with args:
	applies_to_part (String): Filters for work belonging to any of the provided parts.
	owned_by (String): Filters for work owned by any of these users
	created_by (String): Filters for work created by any of these users 
	issue.priority (String): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
    Get all parts containing the word "AI": get_objects_by_keyword
	the ID of the current User: who_am_i
	the current sprint: get_current_sprint
Output:{"result": [("owned_by","who_am_i"),("created_by","MAN-123")],"missing_action": ""}
""",
    """
Directive: Get my issues on all objects made by my Manager
Tool:get_items with args:
	applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
	owned_by (Array[String]): Filters for work owned by any of these users
	created_by (Array[String]): Filters for work created by any of these users 
	issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
    Get all parts containing the word "AI": get_objects_by_keyword
	the ID of the current User: who_am_i
	the current sprint: get_current_sprint
Output:{"result": [], "missing_action": "Get user details of my Manager"}
""",
    """
Directive: Get issues made by users in my team
Tool:get_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
Output:{"result": [],"missing_action": "Get user details of my team"}
""",
    """
Directive: Get issues made by users U-123 and U-456
Tool:get_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
    Get all parts containing the word "AI": get_objects_by_keyword
    the ID of the current User: who_am_i
Output:{"result": [("created_by", ["U-123", "U-456"])],"missing_action": ""}
""",
    """
Directive: Get issues owned by users U-123 and U-456 and with priorities P0 and P1
Tool:get_work_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ["p0", "p1", "p2", "p3"] (optional)
Past Actions/Knowledge:
    Get all parts containing the word "AI": get_objects_by_keyword
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output:{"result": [("created_by", ["U-123", "U-456"]), ("issue.priority":["p0", "p1"])],"missing_action": ""}
""",
    """
Directive: Summarize my open tasks
Tool: summarize_objects with args:
    objects (Array[Object]): The objects to summarize
Past Actions/Knowledge:
Output: {"result": [], "missing_action": "Get my open tasks"}
""",
    """
Directive: Assign a QA tester to Project P-789
Tool: assign_user with args:
    task_key (String): The key of the task to assign
    user_key (Array[String]): The key of the QA tester to assign to the issue
Past Actions/Knowledge:
    List of available QA testers: get_available_qa_testers
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("task_key", "P-789"), ("user_key", ["get_available_qa_testers"])], "missing_action": ""}
""",
    """
Directive: Create a new high-priority issue from action item creted from T in the current backlog
Tool:create_issue with args:
    summary (String): A brief, one-sentence description of the issue
    description (String): A detailed description of the issue
    priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
    backlog (String): The backlog to add the issue to (optional)
Past Actions/Knowledge:
    List of available priorities: get_available_priorities
    Get action items created from T: get_action_items_from_text
    Get Current backlog: get_backlog_issues
    Get the ID of the current User: who_am_i
Output: {"result": [("summary", ""), ("description", ""), ("priority", "High"), ("backlog", "get_backlog_issues")], "missing_action": ""}
""",
    """
Directive: Create an action items created from transcript T
Tool: create_actionable_tasks_from_text with args:
	text (String):The text from which the actionable insights need to be created
Past Actions/Knowledge:
    List of available priorities: get_available_priorities
    Get action items created from T: get_action_items_from_text
    Get Current backlog: get_backlog_issues
    Get the ID of the current User: who_am_i
Output: {"result": [("text", "T")], "missing_action": ""}
""",
    """
Directive: Assign developers to all tasks in the backlog
Tool: assign_users with args:
    tasks (Array[Object]): List of tasks to assign
    users (Array[String]): List of developers to assign to the tasks
Past Actions/Knowledge:
    List of developers: get_available_developers
    List of tasks in the backlog: get_backlog_tasks
    the ID of the current User: who_am_i
Output: {"result": [("tasks", "get_backlog_tasks"), ("users", ["get_available_developers"])], "missing_action": ""}
""",
    """
Directive: Summarize the tasks completed by my team
Tool: summarize_objects with args:
    objects (Array[Object]): The objects to summarize
Past Actions/Knowledge:
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [], "missing_action": "Get completed tasks by my team"}
""",
    """
Directive: Create a new issue with the summary "Fix bug in AI" and the description "This is a bug in AI"
Tool:create_issue with args:
    summary (String): A brief, one-sentence description of the issue
    description (String): A detailed description of the issue
    priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
    backlog (String): The backlog to add the issue to (optional)
Past Actions/Knowledge:
    List of available priorities: get_available_priorities
    Get the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("summary", "Fix bug in AI"), ("description", "This is a bug in AI")], "missing_action": ""}
""",
    """
Directive: Close all unresolved support tickets
Tool: close_issues with args:
    issues (Array[Object]): List of issues to close
Past Actions/Knowledge:
    Get the list of all users: get_all_users
Output: {"result": [], "missing_action": "Get the list of unresolved support tickets"}
""",
    """
Directive: Close all unresolved bugs in the system
Tool: close_issues with args:
    issues (Array[Object]): List of issues to close
    issue.type (String): Filters for issues by the type ["Bug", "Feature", "Optimisation"] (optional)
    issue.status (String): Filters for issues by the status allowing ["Unresolved", "Resolved", "Abandoned"] (optional)
Past Actions/Knowledge:
    Get the list of all bugs: get_all_bugs
Output: {"result": [("issues", ["get_all_bugs"]), ("issue.type", ["Bug"]), ("issue.status", ["Unresolved"])], "missing_action": ""}
""",
    """
Directive: Assign a UI/UX designer to tasks in sprint S-456
Tool: assign_user with args:
    task_key (String): The key of the task to assign
    user_key (Array[String]): The key of the UI/UX designer to assign to the issue
Past Actions/Knowledge:
    List of available UI/UX designers: get_available_workers
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("task_key", "S-456"), ("user_key", ["get_available_workers"])], "missing_action": ""}
""",
    """
Directive: Create a new Blocking issue with the summary "Implement feature X" and the description "Details about implementing feature X"
Tool:create_issue with args:
    summary (String): A brief, one-sentence description of the issue
    description (String): A detailed description of the issue
    priority (String): Priority of the issue allowing: ["High", "Medium", "Low", "Blocker"]
    backlog (String): The backlog to add the issue to (optional)
Past Actions/Knowledge:
    List of available priorities: get_available_priorities
    Get the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Output: {"result": [("summary", "Implement feature X"), ("description", "Details about implementing feature X"), ("priority", "Bloker")], "missing_action": ""}
""",
]

In [298]:
print(len(my_base_examples))

34


In [282]:
import random
some_examples = random.sample(my_base_examples, 10)

l = "\n".join(some_examples)

response = palm.generate_text(
    model=text_model,
    prompt="Generate more 5 examples like the following:\n" + l,
    temperature=0,
    max_output_tokens=800,
)

In [281]:
print(response.result)


 
Directive: Get all tasks created by me
Tool:get_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priority (Array[String]): Filters for issues with any of the provided priorities allowing: ['p0', 'p1', 'p2', 'p3'] (optional)
Past Actions/Knowledge:
    the ID of the current User: who_am_i
    the current sprint: get_current_sprint
Thought: I have all required details.
Output: {"result": [], "missing_action": ""}


Directive: Get all tasks created by me in the current sprint
Tool:get_items with args:
    applies_to_part (Array[String]): Filters for work belonging to any of the provided parts.
    owned_by (Array[String]): Filters for work owned by any of these users
    created_by (Array[String]): Filters for work created by any of these users 
    issue.priorit

In [301]:
def convert_to_csv(fine_tune_examples, file_name):
    with open(file_name, "w") as f:
        for example in fine_tune_examples:
            input = example.split("Output:")[0].strip()
            input = input.replace("\n", "\\n")
            input = input.replace("    ", "\\t")
            output = example.split("Output:")[1].strip()
            f.write("\"" + input + "\",\"" + output + "\"\n")

convert_to_csv(my_base_examples, "my_base_examples.csv")